# Question 1

Q1) (40 points) Write a Python script to perform the tasks in Q1 and Q3 from homework assignment 3
using numpy. Your program should accept the path to the file to read (example:
C:\Documents\random.txt).

In [0]:
import numpy as np
   
#Q1
  
file = "C:\\Documents\\random.txt"
with open(filename, "r") as f:
  csv = np.genfromtxt(f, delimiter=',', dtype=np.unicode_)
  names = csv[0]
  array = csv[1:].astype(np.float64)
  maxes = np.amax(array, axis=0)
  mins = np.amin(array, axis=0)
  means = np.mean(array, axis=0)
  stddevs = np.std(array, axis=0)
  p25,p50,p75 = np.percentile(array,[25, 50, 75], axis=0)
  stats = {"Max":maxes, "Min":mins, "Mean":means, "Stddev":stddevs, "Median":p50, "P25":p25, "P75":p75}  
  
for element in names:
  print('\t' + element, end=' ')
print('\n'+' '*6+'-'*75)

for key in stats.keys():
  print(key, end='\t  ')
  for value in stats.get(key):
    print("%.2f" % (value), end='\t\t ')
  print(end='\n')
print('\n\n')

#Q3

ordered = np.sort(array, axis=0)
col_size = array[:,0].size
y = [0 for x in range(10)]
scales = {names[i]:y for i in range(5)}
keys = list(scales.keys())


normed = np.around((9*((array - mins)/array.ptp(axis=0))), decimals = 0)
print('Scale:' +' '*9, end='')
for i in range(10):
    print(' ' + str(i), end='\t')
print('\n' + '-'*95)

for i in range(array.shape[1]):
  unqArray, cnts = np.unique(normed[:,i], return_counts=True)
  unqs = dict(zip(unqArray, cnts))
  print(names[i], end=":\t")
  for value in unqs.values():
    print(str(value), end = '\t')
  print(end='\n')  
  

	methodA 	methodB 	methodC 	methodD 	methodE 
      ---------------------------------------------------------------------------
Max	  9.00		 14.85		 12.00		 17.42		 9.00		 
Min	  0.00		 -4.33		 0.60		 -11.50		 0.01		 
Mean	  4.58		 5.03		 3.03		 4.61		 4.50		 
Stddev	  2.85		 2.96		 1.51		 4.55		 2.62		 
Median	  4.00		 5.06		 2.71		 4.82		 4.51		 
P25	  2.00		 2.97		 1.97		 1.56		 2.17		 
P75	  7.00		 7.04		 3.68		 7.69		 6.79		 



Scale:          0	 1	 2	 3	 4	 5	 6	 7	 8	 9	
-----------------------------------------------------------------------------------------------
methodA:	78	109	99	112	108	89	93	107	93	112	
methodB:	1	19	74	164	263	268	146	54	9	2	
methodC:	46	376	346	141	47	27	12	2	2	1	
methodD:	2	9	26	111	207	264	230	121	27	3	
methodE:	63	118	106	99	114	109	110	107	126	48	


# Question 2


Q2) (60 points) In this question we will be analyzing the Behavioral Risk Factor Surveillance System
(BRFSS) weight vs. height data. For this analysis, we are interested in five (5) pieces of data: age, current weight (cw),
weight a year ago (waya), height, and gender.
Your program should accept the path to the file to read (example: C:\Documents\CDBRFS08.ASC.gz).
Don’t submit the provided input file (CDBRFS08.ASC.gz) with your notebook.

*   A(10 points): Print first 15 rows of cleaned data, length of data, and datatype of data.
*   B(10 points): Produce summary statistics for cw, waya, and height (mean, standard deviation,
range, and median). Round the final answer to two (2) decimal places
* C(5 points):  How many entries are females younger (<) than 40?
* D(5 points): How many male is within 1 std (<=) in height from the mean of the entire set and from
the mean of male entries?
* E(5 points): How many outlier entries we have for waya for females?
* F(5 points):  Find the coefficient of variation for cw
* G(5 points):  Which group has less variation in cw, male or female? Why? Support your answer
with numbers.
* H(5 points): Do males tend to gain more weight when they are older than (>) 40 compared to
those who are younger?
* I(10 points): Define weight change (delta_w) = cw – waya. Calculate correlation between delta_w
and the following variables, and determine which one is most correlated (regardless of the sign
of correlation) with delta_w:
a) cw, b) waya, c) height, and d) age.



In [13]:
import numpy as np
import gzip

#age, weight (cw), weight last year (waya), height, gender
#clean data by removing any invalid or missing entries
def clean_data(array):
  rows_to_delete = []
  for i in range(array.shape[0]):
    for j in range(array.shape[1]):
    
      try:
        var = int(array[i,j])
      except ValueError:
        rows_to_delete += [i]
        break
        
      if j==0:  #age
        age = var
        if 99 < age or age < 18:
          rows_to_delete += [i]
          break
      
      elif j==1 or j==2:  #weight
        weight = var
        if 9998 >= weight >= 9000:   #kg
          weight = weight-9000
        elif 50 <= weight <= 999:    #lb
          weight = round(weight/2.205, 1)
        else:
          rows_to_delete += [i]
          break
        array[i,j] = weight
      
      elif j==3:  #height
        height = var
        if 9998 >= height >= 9000:   #cm
          height = height- 9000
        elif 711 >= height >= 200:   #ft/inches
          ft= height//100
          inches = height%100
          height = round(ft*30.48 + inches*2.54, 0)
        else:
          rows_to_delete += [i]
          break
        array[i,j] = height
      
      elif j==4:  #gender
        gender = var
        if gender != 1 and gender != 2:
          rows_to_delete += [i]
          break
 
  return np.delete(array, rows_to_delete, axis=0).astype(np.float64)

          

#open file
file = "C:\\Documents\\CDBRFS08.ASC.gz"
ob = gzip.open(file, 'rt')

age=[]
cw=[]
height=[]
waya=[]
gender=[]
for line in ob:
  age += [line[100:102]]
  cw += [line[118:122]] 
  height += [line[122:126]] 
  waya += [line[126:130]] 
  gender += [line[142]] 
   
combined = list(zip(age,cw,waya,height,gender))
array = np.array(combined, ndmin=2)

clean_arr = clean_data(array)
print('Part A\n-------')
print(clean_arr[0:15], len(clean_arr), 'dtype(\'%s\')' % (clean_arr.dtype))
print("-"*95)

print('Part B\n-------')
names = ['age', 'current weight', 'weight a year ago', 'height', 'gender']
for i in [x for x in range(clean_arr.shape[1]) if 4>x>0]:
  print(names[i], end=":\n\t")
  print('mean: %.2f\tstd: %.2f\trange:%.1f\tmedian:%.1f' % 
        (np.mean(clean_arr[:,i]), np.std(clean_arr[:,i]), 
         np.ptp(clean_arr[:,i]), np.median(clean_arr[:,i]))
       )
print("-"*95)
print('Part C\n-------')
young_women = clean_arr[np.logical_and(clean_arr[:,4]==2, clean_arr[:,0]<40)]
print('Number of females under 40: ', len(young_women))
print("-"*95)

print('Part D\n-------')
height = clean_arr[:,3]
height_std = np.std(height)
height_mean = np.mean(height)

males = clean_arr[clean_arr[:,4]==1]
maleheight_std = np.std(males[:,3])
maleheight_mean = np.mean(males[:,3])

#print('maleheight_mean = ', maleheight_mean)
#print('height_mean = ', height_mean)

normal_height_malevworld = males[np.logical_and(
                           males[:,3] >= height_mean-height_std, 
                           males[:,3] <= height_mean+height_std)]
normal_height_malevmale = males[np.logical_and(
                           males[:,3] >= maleheight_mean-maleheight_std, 
                           males[:,3] <= maleheight_mean+maleheight_std)]
print('Number of males within 1 std in height from mean of entire set: ', len(normal_height_malevworld))
print('Number of males within 1 std in height from mean of males: ', len(normal_height_malevmale))
print("-"*95)

print('Part E\n-------')
females = clean_arr[clean_arr[:,4]==2]

waya_female = females[:,2]
waya_female_std = np.std(waya_female)
waya_female_mean = np.mean(waya_female)
waya_female_Q3, waya_female_Q1 = np.percentile(waya_female, [75,  25])
waya_female_IQR = waya_female_Q3-waya_female_Q1

#Outliers are all values above upper fence and below lower fence.

waya_female_outliers = waya_female[np.logical_or(
                       waya_female[:] > waya_female_Q3 + 1.5*waya_female_IQR , 
                       waya_female[:] < waya_female_Q1 - 1.5*waya_female_IQR )] 

print('Number of outlier female entries in waya: ', len(waya_female_outliers))
print("-"*95)

print('Part F\n-------')
cw = clean_arr[:,1]
cw_mean = np.mean(cw)
cw_std = np.std(cw)
cw_cv = (cw_std/cw_mean)*100
print('Coefficient variation (CV) for cw column: %.2f%%' % (cw_cv))
print("-"*95)

print('Part G\n-------')
cw_male = cw[clean_arr[:,4]==1]
cw_female = cw[clean_arr[:,4]==2]

male_cw_variation = np.var(cw_male)
female_cw_variation = np.var(cw_female)
print('females have less variation in cw than males\n',
      '\tmale cw variance: %.2f\n\tfemale cw variance: %.2f' % 
      (male_cw_variation, female_cw_variation))
print("-"*95)

print('Part H\n-------')
ymmeanweight = np.mean(clean_arr[clean_arr[:,0] <= 40][:,1])
ommeanweight = np.mean(clean_arr[clean_arr[:,0] > 40][:,1])
print('Males do not tend to gain more weight as they grow older.\n',
      '\tmean of weight for young (age <= 40) males: %.2fkg\n\tmean of weight for older (age > 40) males: %.2fkg' 
      % (ymmeanweight, ommeanweight))
print("-"*95)

print('Part I\n-------')
age = cw = clean_arr[:,0]
waya = clean_arr[:,2]
delta_w = cw - waya     #weight change
corrs = []
corrs += np.corrcoef(delta_w,age)[0][1], np.corrcoef(delta_w,cw)[0][1], np.corrcoef(delta_w,waya)[0][1], np.corrcoef(delta_w,height)[0][1]


print('the most correlated variable is \'weight a year ago (waya)\' because it has the highest absolute value.',
      '\nIt\'s likely the most correlated since a person\'s weight 1yr ago is closely related to the change in weight they\'ve had over the course of a year.')
for i in range(4):
  print('\tCorrelation coefficient delta and %s: %.2f' % (names[i], corrs[i]))


Part A
-------
[[ 39.   88.4  88.4 180.    1. ]
 [ 64.   74.8  84.4 155.    2. ]
 [ 51.   99.8  99.8 183.    1. ]
 [ 35.   63.5  61.2 170.    2. ]
 [ 62.   70.3  70.3 173.    2. ]
 [ 64.   63.5  63.5 157.    2. ]
 [ 55.   82.5  82.5 155.    2. ]
 [ 71.   59.   56.7 155.    2. ]
 [ 21.   81.6  86.2 180.    1. ]
 [ 45.   90.7  90.7 165.    2. ]
 [ 53.   51.7  51.7 163.    2. ]
 [ 51.   56.7  59.  155.    2. ]
 [ 70.  105.  112.  170.    2. ]
 [ 59.   83.9  83.9 165.    2. ]
 [ 59.  131.  129.  190.    1. ]] 385974 dtype('float64')
-----------------------------------------------------------------------------------------------
Part B
-------
current weight:
	mean: 78.83	std: 19.37	range:279.0	median:77.1
weight a year ago:
	mean: 79.57	std: 20.42	range:318.3	median:77.1
height:
	mean: 169.00	std: 10.36	range:175.0	median:168.0
-----------------------------------------------------------------------------------------------
Part C
-------
Number of females under 40:  47153
-------------------